In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [2]:
pd.set_option('display.max_rows', 6)

In [3]:
df = pd.read_csv('user_taggedartists-timestamps.dat', sep='\t')
df

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
...,...,...,...,...
186476,2100,16437,2087,1277935200000
186477,2100,16437,2801,1272664800000
186478,2100,16437,3335,1277935200000


In [4]:
df1 = df[df["timestamp"]>294357600000]
df1.min()

userID                   2
artistID                 1
tagID                    1
timestamp    1122847200000
dtype: int64

In [5]:
df1[["timestamp"]] = df1[["timestamp"]]-1122847200000
df1

e:\language\python37\lib\site-packages\pandas\core\frame.py:3489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,userID,artistID,tagID,timestamp
0,2,52,13,115689600000
1,2,52,15,115689600000
2,2,52,18,115689600000
...,...,...,...,...
186476,2100,16437,2087,155088000000
186477,2100,16437,2801,149817600000
186478,2100,16437,3335,155088000000


In [8]:
uit_df = df1[["userID","artistID","timestamp"]]
sort_uit_df = uit_df.sort_values(by=['userID','timestamp'],ascending=[True,True]) 
sort_uit_df = sort_uit_df.drop_duplicates()
sort_uit_df[["timestamp"]] = (sort_uit_df[["timestamp"]]/1000000000).astype("int")

In [17]:
# 去除交互少于N的user和item
ui_cnt = sort_uit_df['userID'].value_counts()
u_index = ui_cnt[ui_cnt>=4].index
ucut_df = sort_uit_df[sort_uit_df['userID'].isin(u_index)]
ucut_df.nunique()

userID        1407
artistID     12373
timestamp       70
dtype: int64

In [18]:
ucut_df = ucut_df.drop_duplicates()
ucut_df.nunique()

userID        1407
artistID     12373
timestamp       70
dtype: int64

In [19]:
utgroup = ucut_df.groupby(['userID','timestamp'])
ut_c = utgroup.userID.count()[lambda x: x >= 3]
utc_id = ut_c.index.levels
utc_id_index = ut_c.index.codes


user_session = {}
L = len(utc_id_index[0])
for i in range(L): 
    loc_i = utc_id_index[0][i]
    loc_j = utc_id_index[1][i]
    uid = utc_id[0][loc_i]
    tid = utc_id[1][loc_j]
    if uid not in user_session:
        user_session[uid] = []
    if len(user_session[uid])<=8:
        user_session[uid].append(tid)
    
len(user_session)


1211

In [21]:
df_cut=pd.DataFrame(columns=ucut_df.keys())
df_cut

,userID,artistID,timestamp


In [22]:
cnt = 0
short_c = 0
for uid in user_session: 
#     if len(user_session[uid])<2:
#         short_c+=1
#         continue
    cnt+=1
    if cnt%1000==0:
        print("cnt = ",cnt)
    tid = user_session[uid]
    df_oneuser = ucut_df[ucut_df["userID"]==uid]
#     df_oneuser
    df_row = df_oneuser[df_oneuser["timestamp"].isin(tid)]
    df_cut = pd.concat([df_cut, df_row],ignore_index=True)
    

cnt =  1000


In [23]:
df_cut.nunique()

userID        1211
artistID     10802
timestamp       70
dtype: int64

In [ ]:
df4.to_csv('LastFM_cut.csv',index=0)

In [ ]:
ui_array = df4[['userID','artistID']].values
uid_turn = defaultdict(int)
iid_turn = defaultdict(int)
uid_cnt = 0
# uid_cnt = 1
# 因为用了np.zero生产数据,第0行也要有实际数据,所以uid要从0开始
# maskid是0，对应的是item，userid可以是0
iid_cnt = 1
for row in ui_array:
    uid = row[0]
    iid = row[1]
    if uid not in uid_turn:
        uid_turn[uid] = uid_cnt
        uid_cnt += 1
    if iid not in iid_turn:
        iid_turn[iid] = iid_cnt
        iid_cnt += 1

len(uid_turn)
len(iid_turn)

In [ ]:
uit_value = df4.values
userList = {}

for row in uit_value:
    uid = uid_turn[row[0]]
    iid = iid_turn[row[1]]

    tid = row[2]
    if uid not in userList:
        userList[uid] = []
        
    userList[uid].append([iid,tid])



In [ ]:
ut_List = {}
for uid in userList:
    it_list = userList[uid]
    sessionList = {}
    for row in it_list:
        
        iid = row[0]
        tid = row[1]
        if tid not in sessionList:
            sessionList[tid] = []

        sessionList[tid].append(iid)
    
    ut_List[uid] = sessionList
        

In [ ]:
u_List = {}
for uid in ut_List:
    u_List[uid] = []
    for tid in ut_List[uid]:
        u_List[uid].append(ut_List[uid][tid])

In [ ]:
json_u_List = json.dumps(u_List)
saveFile(json_u_List,'LastFM.json')